# Gaining insights from Unstructured Data with Snowflake Cortex AI

Tasty Bytes is a global food truck network operating in 15 countries with fleet of 450 trucks. They collect customer reviews to get customer feedback on their food-trucks which come in from multiple sources and span multiple languages. 
This enables them to better understand the areas which require improvement and drive up customer loyalty along with satisfaction. 

In this notebook, we will look at how we analyze these collated customer reviews using Snowflake Cortex to understand :
  * What our international customers are saying with Cortex **Translate**
  * Get a summary of what customers are saying with Cortex **Summary**
  * Classify reviews to determine if they would recommend a food truck with Cortex **ClassifyText**
  * Gain specific insights with Cortex **Complete**
  * Understand how customers are feeling with Cortex **Sentiment**

Lets see how many reviews we have.

In [ ]:
SELECT COUNT(*) FROM TRUCK_REVIEWS_V;

**Import python packages**

Snowflake Notebooks include Streamlit and the third-party packages listed in the Snowflake Anaconda channel.  
Installing a package is made easy by enabling user to select required pacakges from a list of available pacakges under Packages on the top right corner.  
Once installed, we can import installed packages as we would in any other notebook.

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# Snowpark
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F
from snowflake.snowpark.functions import when, date_part
from snowflake.snowpark.window import Window

# Cortex Functions
import snowflake.cortex  as cortex

session = get_active_session()
# Add a query tag to the session.
session.query_tag = {"origin":"sf_sit-is", 
                     "name":"voc", 
                     "version":{"major":1, "minor":0},
                     "attributes":{"is_quickstart":1, "source":"notebook", "vignette":"customer_reviews"}}


Lets preview the reviews

In [ ]:
reviews_df = session.table('TRUCK_REVIEWS_V') \
             .filter(date_part("year", F.col('DATE')) == 2024)
reviews_df

In the next cell, we leverage **Translate** - one of the **Snowflake Cortex specialised LLM functions** are available in Snowpark ML, to translate the multilingual reviews to english to enable easier analysis for folks who don't speak the language the original review is in.

In [ ]:
# Conditionally translate reviews that are not english using Cortex Translate
reviews_df = reviews_df.withColumn('TRANSLATED_REVIEW',when(F.col('LANGUAGE') != F.lit("en"), \
                                                            cortex.translate(F.col('REVIEW'), \
                                                                             F.col('LANGUAGE'), \
                                                                             "en")) \
                                   .otherwise(F.col('REVIEW')))

reviews_df.filter(F.col('LANGUAGE') != F.lit("en")) \
.select(["REVIEW","LANGUAGE","TRANSLATED_REVIEW"]).show(3)

We can quickly learn what our customers are saying with Cortex **Summarize**

In [ ]:
summarized_reviews_df = session.table("CONCATENATED_REVIEWS").select(
    F.col("TRUCK_BRAND_NAME"),
    cortex.summarize(F.col("ALL_REVIEWS_TEXT")).alias("SUMMARY")
)

summarized_reviews_df.select(["TRUCK_BRAND_NAME", "SUMMARY"]).show(3)

We can similarly understand if a customer would recommend the food truck based on their review using Snowflake Cortex **ClassifyText**

In [ ]:
# Text description to understand whether a customer would recommend food truck based on their review 
text_description = """
Tell me based on the following food truck customer review, will they recommend the food truck to \
their friends and family?
"""

reviews_df = reviews_df.withColumn('RECOMMEND', cortex.ClassifyText(F.col('REVIEW'),["Likely","Unlikely","Unsure"], text_description)['label'])

reviews_df.select(["REVIEW","RECOMMEND"]).show(3)

Gaining specific insights through Cortex Complete

In [ ]:
question = "What is the number one dish positively mentioned in the feedback?"

summarized_reviews_df = session.table("CONCATENATED_REVIEWS").select(
    F.col("TRUCK_BRAND_NAME"),
    cortex.complete(
        "mistral-large2",
        F.concat(
            F.lit("Context: "),
            F.col("ALL_REVIEWS_TEXT"),
            F.lit(f" Question: {question} Answer briefly and concisely and only name the dish:")
        )
    ).alias("NUMBER_ONE_DISH")
)

summarized_reviews_df.show(3)

Next we will look at another **task specific LLM function in Cortex - Sentiment**. We utilise sentiment function to understand customer's tone based on the review they provided. Sentiment return value between -1 and 1 such that -1 is the most negative while 1 is the most positive.  

In [ ]:
# Understand the sentiment of customer review using Cortex Sentiment
reviews_df = reviews_df.withColumn('SENTIMENT', cortex.sentiment(F.col('REVIEW')))

reviews_df.select(["REVIEW","SENTIMENT"])

These robust Cortex functions seamlessly integrate with SQL, enabling powerful data processing capabilities.

In [ ]:
-- Add the TRANSLATED_REVIEW column with conditional translation
WITH TRANSLATED_REVIEWS AS (
    SELECT 
        REVIEW,
        LANGUAGE,
        CASE 
            WHEN LANGUAGE != 'en' THEN SNOWFLAKE.CORTEX.TRANSLATE(REVIEW, LANGUAGE, 'en') 
            ELSE REVIEW
        END AS TRANSLATED_REVIEW
    FROM TRUCK_REVIEWS_V
)

-- Filter rows where the LANGUAGE is not English and select the desired columns
SELECT 
    REVIEW, 
    LANGUAGE, 
    TRANSLATED_REVIEW
FROM TRANSLATED_REVIEWS
WHERE LANGUAGE != 'en'
LIMIT 3;

In [ ]:
-- Generate summaries for each truck brand
WITH SUMMARIZED_REVIEWS AS (
    SELECT 
        TRUCK_BRAND_NAME,
        SNOWFLAKE.CORTEX.SUMMARIZE(ALL_REVIEWS_TEXT) AS SUMMARY
    FROM CONCATENATED_REVIEWS
)

SELECT * FROM SUMMARIZED_REVIEWS;

In [ ]:
WITH CLASSIFIED_REVIEWS AS (
    SELECT 
        REVIEW,
        PARSE_JSON(SNOWFLAKE.CORTEX.CLASSIFY_TEXT(
            REVIEW, 
            ['Likely', 'Unlikely', 'Unsure'], 
            OBJECT_CONSTRUCT('task_description', 
                'Tell me based on the following food truck customer review, will they recommend the food truck to their friends and family?'
            )
        )):label::TEXT AS RECOMMEND
    FROM TRUCK_REVIEWS_V
)

SELECT * From CLASSIFIED_REVIEWS limit 3;


In [ ]:
-- Gain Learnings from a specific question
WITH GAIN_LEARNINGS AS (
    SELECT 
        TRUCK_BRAND_NAME,
        SNOWFLAKE.CORTEX.COMPLETE(
           'mistral-large2', 
           'Context:' || ALL_REVIEWS_TEXT || ' Question: What is the number one dish positively mentioned in the feedback? Answer briefly and concisely and only name the dish:'
       ) AS NUMBER_ONE_DISH
    FROM CONCATENATED_REVIEWS
)
SELECT TRUCK_BRAND_NAME, NUMBER_ONE_DISH FROM GAIN_LEARNINGS LIMIT 3;

In [ ]:
SELECT 
    REVIEW, 
    SNOWFLAKE.CORTEX.SENTIMENT(REVIEW) AS SENTIMENT
FROM TRUCK_REVIEWS_V
LIMIT 3;


StreamLit Application

In [ ]:
# Import python packages
import pandas as pd
import streamlit as st
import snowflake.cortex  as cortex
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T

# Set Streamlit page configuration
st.set_page_config(layout="wide", initial_sidebar_state="expanded")
st.title(":truck: Food Truck AI Support")
st.caption(
    f"""Welcome! This application suggests answers to customer questions based 
    on corporate documentation and previous agent responses in support chats.
    """
)

# Get current credentials
session = get_active_session()

# Constants
CHAT_MEMORY = 20
DOC_TABLE = "app.vector_store"


# Reset chat conversation
def reset_conversation():
    st.session_state.messages = [
        {
            "role": "assistant",
            "content": "What question do you need assistance answering?",
        }
    ]




##########################################
#       RAG
##########################################
def get_context(chat, DOC_TABLE):
    chat_summary = summarize(chat)
    return find_similar_doc(chat_summary, DOC_TABLE)


def summarize(chat):
    summary = cortex.complete(
        "mistral-large",
        "Provide the most recent question with essential context from this support chat: "
        + chat,
    )
    return summary.replace("'", "")


def find_similar_doc(text, DOC_TABLE):
    doc = session.sql(f"""Select input_text,
                        source_desc,
                        VECTOR_COSINE_SIMILARITY(chunk_embedding, SNOWFLAKE.CORTEX.EMBED_TEXT_768('e5-base-v2', '{text.replace("'", "''")}')) as dist
                        from {DOC_TABLE}
                        order by dist desc
                        limit 1
                        """).to_pandas()
    st.info("Selected Source: " + doc["SOURCE_DESC"].iloc[0])
    return doc["INPUT_TEXT"].iloc[0]

##########################################
#       Prompt Construction
##########################################
if "background_info" not in st.session_state:
    st.session_state.background_info = (
        session.table("app.documents")
        .select("raw_text")
        .filter(F.col("relative_path") == "tasty_bytes_who_we_are.pdf")
        .collect()[0][0]
    )


def get_prompt(chat, context):
    prompt = f"""Answer this new customer question sent to our support agent
        at Tasty Bytes Food Truck Company. Use the background information
        and provided context taken from the most relevant corporate documents
        or previous support chat logs with other customers.
        Be concise and only answer the latest question.
        The question is in the chat.
        Chat: <chat> {chat} </chat>.
        Context: <context> {context} </context>.
        Background Info: <background_info> {st.session_state.background_info} </background_info>."""
    return prompt.replace("'", "")


##########################################
#       Chat with LLM
##########################################
if "messages" not in st.session_state:
    reset_conversation()

if user_message := st.chat_input():
    st.session_state.messages.append({"role": "user", "content": user_message})

for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

if st.session_state.messages[-1]["role"] != "assistant":
    chat = str(st.session_state.messages[-CHAT_MEMORY:]).replace("'", "")
    with st.chat_message("assistant"):
        with st.status("Answering..", expanded=True) as status:
            st.write("Finding relevant documents & support chat logs...")
            # Get relevant information
            context = get_context(chat, DOC_TABLE)
            st.write("Using search results to answer your question...")
            # Ask LLM
            prompt = get_prompt(chat, context)
            response = cortex.complete("mistral-large", prompt)
            status.update(label="Complete!", state="complete", expanded=False)
        st.markdown(response)
    st.session_state.messages.append({"role": "assistant", "content": response})

In [ ]:
USE ROLE sysadmin;

  /*--
  • database, schema and warehouse creation
  --*/

  -- create tb_voc database
  CREATE OR REPLACE DATABASE tb_voc;

  -- create raw_pos schema
  CREATE OR REPLACE SCHEMA tb_voc.raw_pos;

  -- create raw_customer schema
  CREATE OR REPLACE SCHEMA tb_voc.raw_support;

  -- create harmonized schema
  CREATE OR REPLACE SCHEMA tb_voc.harmonized;

  -- create analytics schema
  CREATE OR REPLACE SCHEMA tb_voc.analytics;

  -- create tasty_ds_wh warehouse
  CREATE OR REPLACE WAREHOUSE tasty_ds_wh
      WAREHOUSE_SIZE = 'large'
      WAREHOUSE_TYPE = 'standard'
      AUTO_SUSPEND = 60
      AUTO_RESUME = TRUE
      INITIALLY_SUSPENDED = TRUE
  COMMENT = 'data science warehouse for tasty bytes';


  USE WAREHOUSE tasty_ds_wh;

  /*--
  • file format and stage creation
  --*/

  CREATE OR REPLACE FILE FORMAT tb_voc.public.csv_ff 
  type = 'csv';

  CREATE OR REPLACE STAGE tb_voc.public.s3load
  COMMENT = 'Quickstarts S3 Stage Connection'
  url = 's3://sfquickstarts/tastybytes-voc/'
  file_format = tb_voc.public.csv_ff;

  /*--
  raw zone table build 
  --*/

  -- menu table build
  CREATE OR REPLACE TABLE tb_voc.raw_pos.menu
  (
      menu_id NUMBER(19,0),
      menu_type_id NUMBER(38,0),
      menu_type VARCHAR(16777216),
      truck_brand_name VARCHAR(16777216),
      menu_item_id NUMBER(38,0),
      menu_item_name VARCHAR(16777216),
      item_category VARCHAR(16777216),
      item_subcategory VARCHAR(16777216),
      cost_of_goods_usd NUMBER(38,4),
      sale_price_usd NUMBER(38,4),
      menu_item_health_metrics_obj VARIANT
  );

  -- truck table build 
  CREATE OR REPLACE TABLE tb_voc.raw_pos.truck
  (
      truck_id NUMBER(38,0),
      menu_type_id NUMBER(38,0),
      primary_city VARCHAR(16777216),
      region VARCHAR(16777216),
      iso_region VARCHAR(16777216),
      country VARCHAR(16777216),
      iso_country_code VARCHAR(16777216),
      franchise_flag NUMBER(38,0),
      year NUMBER(38,0),
      make VARCHAR(16777216),
      model VARCHAR(16777216),
      ev_flag NUMBER(38,0),
      franchise_id NUMBER(38,0),
      truck_opening_date DATE
  );

  -- order_header table build
  CREATE OR REPLACE TABLE tb_voc.raw_pos.order_header
  (
      order_id NUMBER(38,0),
      truck_id NUMBER(38,0),
      location_id FLOAT,
      customer_id NUMBER(38,0),
      discount_id VARCHAR(16777216),
      shift_id NUMBER(38,0),
      shift_start_time TIME(9),
      shift_end_time TIME(9),
      order_channel VARCHAR(16777216),
      order_ts TIMESTAMP_NTZ(9),
      served_ts VARCHAR(16777216),
      order_currency VARCHAR(3),
      order_amount NUMBER(38,4),
      order_tax_amount VARCHAR(16777216),
      order_discount_amount VARCHAR(16777216),
      order_total NUMBER(38,4)
  );

  -- truck_reviews table build
  CREATE OR REPLACE TABLE tb_voc.raw_support.truck_reviews
  (
      order_id NUMBER(38,0),
      language VARCHAR(16777216),
      source VARCHAR(16777216),
      review VARCHAR(16777216),
      review_id NUMBER(18,0)
  );

  /*--
  • harmonized view creation
  --*/

  -- truck_reviews_v view
  CREATE OR REPLACE VIEW tb_voc.harmonized.truck_reviews_v
      AS
  SELECT DISTINCT
      r.review_id,
      r.order_id,
      oh.truck_id,
      r.language,
      source,
      r.review,
      t.primary_city,
      oh.customer_id,
      TO_DATE(oh.order_ts) AS date,
      m.truck_brand_name
  FROM tb_voc.raw_support.truck_reviews r
  JOIN tb_voc.raw_pos.order_header oh
      ON oh.order_id = r.order_id
  JOIN tb_voc.raw_pos.truck t
      ON t.truck_id = oh.truck_id
  JOIN tb_voc.raw_pos.menu m
      ON m.menu_type_id = t.menu_type_id;

  /*--
  • analytics view creation
  --*/

  -- truck_reviews_v view
  CREATE OR REPLACE VIEW tb_voc.analytics.truck_reviews_v
      AS
  SELECT * FROM harmonized.truck_reviews_v;


  /*--
  raw zone table load 
  --*/


  -- menu table load
  COPY INTO tb_voc.raw_pos.menu
  FROM @tb_voc.public.s3load/raw_pos/menu/;

  -- truck table load
  COPY INTO tb_voc.raw_pos.truck
  FROM @tb_voc.public.s3load/raw_pos/truck/;

  -- order_header table load
  COPY INTO tb_voc.raw_pos.order_header
  FROM @tb_voc.public.s3load/raw_pos/order_header/;

  -- truck_reviews table load
  COPY INTO tb_voc.raw_support.truck_reviews
  FROM @tb_voc.public.s3load/raw_support/truck_reviews/;


  -- scale wh to medium
  ALTER WAREHOUSE tasty_ds_wh SET WAREHOUSE_SIZE = 'Medium';


  CREATE OR REPLACE TABLE CONCATENATED_REVIEWS AS
  WITH RANKED_REVIEWS AS (
      SELECT 
          TRUCK_BRAND_NAME,
          REVIEW,
          ROW_NUMBER() OVER (PARTITION BY TRUCK_BRAND_NAME ORDER BY REVIEW) AS ROW_NUM
      FROM TRUCK_REVIEWS_V
  ),
  FILTERED_REVIEWS AS (
      SELECT *
      FROM RANKED_REVIEWS
      WHERE ROW_NUM <= 20
  ),
  AGGREGATED_REVIEWS AS (
      SELECT 
          TRUCK_BRAND_NAME,
          ARRAY_AGG(REVIEW) AS ALL_REVIEWS
      FROM FILTERED_REVIEWS
      GROUP BY TRUCK_BRAND_NAME
  ),
  CONCATENATED_REVIEWS AS (
      SELECT 
          TRUCK_BRAND_NAME,
          ARRAY_TO_STRING(ALL_REVIEWS, ' ') AS ALL_REVIEWS_TEXT
      FROM AGGREGATED_REVIEWS
  )

  SELECT * FROM CONCATENATED_REVIEWS;

-- setup completion note
SELECT 'Setup is complete' AS note;





-- Database
CREATE OR REPLACE DATABASE tasty_bytes_chatbot;

--Schema
CREATE OR REPLACE SCHEMA tasty_bytes_chatbot.app;

--Warehouse
CREATE OR REPLACE WAREHOUSE tasty_bytes_chatbot_wh with
WAREHOUSE_SIZE = LARGE
AUTO_SUSPEND = 60;

CREATE OR REPLACE FILE FORMAT tasty_bytes_chatbot.app.csv_ff 
TYPE = 'csv';

CREATE OR REPLACE STAGE tasty_bytes_chatbot.app.s3load
COMMENT = 'Quickstarts S3 Stage Connection'
url = 's3://sfquickstarts/tastybytes-cx/app/'
file_format = tasty_bytes_chatbot.app.csv_ff;

CREATE OR REPLACE TABLE tasty_bytes_chatbot.app.documents (
	RELATIVE_PATH VARCHAR(16777216),
	RAW_TEXT VARCHAR(16777216)
)
COMMENT = '{"origin":"sf_sit-is", "name":"voc", "version":{"major":1, "minor":0}, "attributes":{"is_quickstart":1, "source":"streamlit", "vignette":"rag_chatbot"}}';

COPY INTO tasty_bytes_chatbot.app.documents
FROM @tasty_bytes_chatbot.app.s3load/documents/;

-- https://docs.snowflake.com/en/sql-reference/data-types-vector#loading-and-unloading-vector-data
CREATE OR REPLACE TABLE tasty_bytes_chatbot.app.array_table (
  SOURCE VARCHAR(6),
	SOURCE_DESC VARCHAR(16777216),
	FULL_TEXT VARCHAR(16777216),
	SIZE NUMBER(18,0),
	CHUNK VARCHAR(16777216),
	INPUT_TEXT VARCHAR(16777216),
	CHUNK_EMBEDDING ARRAY
);

COPY INTO tasty_bytes_chatbot.app.array_table
FROM @tasty_bytes_chatbot.app.s3load/vector_store/;

CREATE OR REPLACE TABLE tasty_bytes_chatbot.app.vector_store (
	SOURCE VARCHAR(6),
	SOURCE_DESC VARCHAR(16777216),
	FULL_TEXT VARCHAR(16777216),
	SIZE NUMBER(18,0),
	CHUNK VARCHAR(16777216),
	INPUT_TEXT VARCHAR(16777216),
	CHUNK_EMBEDDING VECTOR(FLOAT, 768)
) AS
SELECT 
  source,
	source_desc,
	full_text,
	size,
	chunk,
	input_text,
  chunk_embedding::VECTOR(FLOAT, 768)
FROM tasty_bytes_chatbot.app.array_table;



